In [1]:
import numpy as np
import gurobipy as grb
import csv
import time
import statsmodels.api as sm
from joblib import Parallel,delayed

def five_variate(n,SNR,rho):
    beta1 = np.array([1,1,1,0,1])
    beta2 = np.array([-1,-1,-1,0,-1])
    p = len(beta1)
    Sigma = np.zeros((p,p));
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**(abs(i-j))
    X = np.random.multivariate_normal(mean =np.zeros(p), cov = Sigma, size = n)
    noise1 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta1)/SNR), size = int(n/2)) 
    noise2 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta2)/SNR), size = int(n/2)) 
    Y1 = X[:int(n/2)]@beta1 + noise1
    Y2 = X[int(n/2):]@beta2 + noise2
    Y = np.r_[Y1,Y2]
    lam = 0.1*n
    ols = sm.OLS(Y, X)
    ols_result = ols.fit()
    bigM = 10*float(1.96*(sum(ols_result.bse)))#1000

    M = grb.Model()
    M.Params.TimeLimit = 400
    beta = M.addMVar((int(n),5), lb = -grb.GRB.INFINITY)
    z = M.addMVar(len(Y) - 1, vtype = grb.GRB.BINARY)
    sum_of_sq = sum((Y[t] - sum(X[t,j]*beta[t,j] for j in range(5)))* (Y[t] - sum(X[t,j]*beta[t,j] for j in range(5))) for t in range(len(Y)))
    for t in range(len(Y)-1):
        for j in range(5):
            M.addConstr(beta[t+1,j] - beta[t,j] <= bigM*z[t])
            M.addConstr(beta[t+1,j] - beta[t,j] >= -bigM*z[t])
    #optional constratins to add more cuts
    M.addConstr(sum(z) <= 5)
    for t in range(1, len(Y) - 2):
        M.addConstr(z[t-1] + z[t] + z[t+1] <= 1)
    # l1-reg
    # reference: https://support.gurobi.com/hc/en-us/community/posts/360077266711-C-sum-absolute-value-in-objective
    phi = 0.0005*n
    beta_abs = M.addMVar((int(n),5)) 
    M.addConstrs(beta_abs[t, j] == grb.abs_(beta[t,j]) for j in range(5) for t in range(len(Y)))# compute absolute value for each beta
    l1_penalty =  phi*sum(sum(beta_abs[t, j] for j in range(5)) for t in range(len(Y))) # compute l1 penalty term
    M.setObjective(sum_of_sq+ lam*sum(z) + l1_penalty)
    M.optimize()
    beta_hat = [beta[t].x for t in range(len(Y))]
    z_hat = [z[t].x for t in range(len(Y)-1)]
    opt_gap = M.MIPGap
    return z_hat, opt_gap,beta_hat

SNR_list = np.array([6, 3.52, 2.07, 1.22, 0.71])
rho_list = np.array([0, 0.3, 0.7])
n_list = np.array([1000])

header = ['Repitition','Time','rho', 'SNR', 'n', 'Optimality Gap', 'Change Points']#, 'beta_hat', 'nonzero count']
f = open('simulation_five_variate_l1_reg.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
f.close()

def to_repeat(rep):
    for n in n_list:
        for SNR in SNR_list:
            for rho in rho_list:
                tik = time.time()
                z_hat, opt_gap, beta_hat = five_variate(n,SNR,rho)
                tok = time.time()
                duration = tok - tik
                cps = [idx for idx,item in enumerate(z_hat) if item == 1 ]
                # Write result
                resultrow = [rep,duration,rho,SNR, n, opt_gap, cps]
                f = open('simulation_five_variate_l1_reg.csv', 'a')
                writer = csv.writer(f)
                writer.writerow(resultrow)
                f.close()
                
Parallel(n_jobs = 8)(delayed(to_repeat)(rep) for rep in range(50)) #Tomo's machine has 8 cores.

Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   10377    3.5838530e+03   0.000000e+00   4.585536e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11814    3.4608745e+03   0.000000e+00   4.339617e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   10698    3.3890938e+03   0.000000e+00   4.270240e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   10393    3.7200226e+03   0.000000e+00   4.603454e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11018    3.4581552e+03   0.000000e+00   4.285347e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11112    3.4319455e+03   0.000000e+00   4.417756e+03      5s
   16843    3.0630201e+03   0.000000e+00   3.193674e+03     10s
   15901    3.1994128e+03   0.000000


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2283.97603    0  179 4370.36043 2283.97603  47.7%     -   52s
     0     0 2309.18811    0  191 4483.62461 2309.18811  48.5%     -   52s
     0     0 2302.10360    0  193 4456.11408 2302.10360  48.3%     -   53s
     0     0 2285.91693    0  179 4370.36043 2285.91693  47.7%     -   53s
     0     0 2383.19985    0  202 4930.97413 2383.19985  51.7%     -   53s
     0     2 2244.74040    0  167 4462.73957 2244.74040  49.7%     -   64s
     1     4 2244.74040    1  167 4462.73957 2244.74040  49.7%  47.0   66s
     0     2 2344.16622    0  176 4792.15642 2344.16622  51.1%     -   66s
     0     2 2285.43449    0  177 4541.80912 2285.43449  49.7%     -   66s
     0     2 2309.18811    0  191 4483.62461 2309.18811  48.5%     -   68s
     7    12 2245.39387    3  165 4462.73957 2245.39387  49.7%   262   70s
     7    12 2348.00482

    55    61 2309.38254   15  200 4061.45427 2308.20053  43.2%  77.0  180s
   119    99 2335.72216   26  187 2481.99971 2297.29847  7.44%   111  180s
    93    81 2394.29917   22  206 2530.46065 2344.81632  7.34%   112  180s
    58    38 2383.88791   16  187 2520.67839 2358.36814  6.44%   216  181s
    53    54 2320.37120   14  195 2561.93638 2316.65128  9.57%   103  180s
    57    37 2346.62418   15  184 2442.39753 2297.56697  5.93%   169  182s
    54    53 2460.93088   14  203 2570.28190 2399.83690  6.63%   151  184s
   102    88 2395.27282   24  205 2530.46065 2344.81632  7.34%   106  185s
    66    72 2309.92147   17  203 4061.45427 2308.20053  43.2%  73.1  187s
    62    38 2350.64596   17  184 2442.39753 2297.56697  5.93%   178  187s
    63    40 2384.68280   18  190 2520.67839 2358.36814  6.44%   214  187s
   140   131 2336.69927   30  187 2481.99971 2297.29847  7.44%   100  187s
    60    39 2257.32557   17  163 2402.01451 2248.44954  6.39%   182  188s
    71    80 2310.15202  

   442   220 2277.76843   62  176 2402.01451 2248.44954  6.39%  63.8  253s
   269   236 2474.52389   64  214 2570.28190 2399.83690  6.63%  72.6  252s
   243   225 2352.45950   53  188 2561.93638 2316.65128  9.57%  71.4  252s
   600   477 2471.30607  124  216 2530.46065 2344.81632  7.34%  44.8  254s
   331   150 2378.19958   63  161 2442.39753 2297.56697  5.93%  74.3  254s
   308   156 2410.19657   50  198 2520.67839 2358.36814  6.44%  89.1  255s
   298   322 2328.55013   70  192 4061.45427 2308.20053  43.2%  46.0  255s
   467   241 2278.21474   67  174 2402.01451 2248.44954  6.39%  62.4  258s
   617   554 2400.47693  159  194 2481.99971 2297.29847  7.44%  49.8  257s
   292   249 2475.80812   67  213 2570.28190 2399.83690  6.63%  69.7  257s
   272   251 2354.44981   59  187 2561.93638 2316.65128  9.57%  68.1  257s
   370   164 2380.66200   73  157 2442.39753 2297.56697  5.93%  70.3  258s
   649   501 2477.34455  133  208 2530.46065 2344.81632  7.34%  43.3  259s
   336   164 2418.52417  

...

   275   312 5389.05420   55  292 6974.41492 5262.71081  24.5%  87.4  170s
   341   367 5426.66122   71  290 6974.41492 5262.71081  24.5%  82.4  177s
   366   401 5433.94725   75  292 6974.41492 5262.71081  24.5%  81.9  181s
   400   435 5450.34106   82  297 6974.41492 5262.71081  24.5%  79.9  185s
   463   502 5479.65208   94  313 6974.41492 5262.71081  24.5%  79.2  194s
   501   536 5492.69392  105  318 6974.41492 5262.71081  24.5%  77.3  198s
   535   591 5502.45990  114  315 6974.41492 5262.71081  24.5%  75.8  202s
   201   210 5963.73028   38  283 8569.01022 5840.33893  31.8%  81.0  208s
   209   246 5966.81604   40  288 8569.01022 5840.33893  31.8%  79.6  210s
   590   640 5519.14230  124  316 6974.41492 5262.71081  24.5%  74.5  214s
   319   351 6017.20720   68  294 8569.01022 5840.33893  31.8%  66.4  217s
   380   428 6034.29993   80  285 8569.01022 5840.33893  31.8%  64.0  222s
   639   703 5533.40138  136  312 6974.41492 5262.71081  24.5%  72.1  225s
   464   517 6071.17473  

     0     0 7794.83633    0  271 15373.0457 7794.83633  49.3%     -   26s
     0     0 7060.01489    0  285 13781.5146 7060.01489  48.8%     -   26s
     0     2 7794.83633    0  271 15373.0457 7794.83633  49.3%     -   35s
     0     2 7060.01489    0  285 13781.5146 7060.01489  48.8%     -   35s
     3     8 7074.21101    2  292 13781.5146 7074.21101  48.7%   482   40s
     7    12 7818.38546    3  268 15373.0457 7818.38546  49.1%   591   41s
    15    20 7098.79470    5  299 13781.5146 7093.65671  48.5%   316   45s
    19    24 7840.48184    6  269 15373.0457 7820.31190  49.1%   282   46s
    27    32 7845.90990    8  274 15373.0457 7820.31190  49.1%   215   97s
H   28    32                    12618.627176 7820.31190  38.0%   207   97s
    35    40 7855.30526   10  276 12618.6272 7820.31190  38.0%   182  100s
    27    32 7119.32812    8  308 13781.5146 7093.65671  48.5%   228  100s
H   30    32                    12375.862243 7093.65671  42.7%   211  100s
    49    55 7869.18541  

Set parameter TimeLimit to value 400
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10988 rows, 10999 columns and 33960 nonzeros
Model fingerprint: 0xb37706c4
Model has 15000 quadratic objective terms
Model has 5000 general constraints
Variable types: 10000 continuous, 999 integer (999 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-04, 1e+02]
  QObjective range [2e-08, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 24446.501447
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10988 rows, 10999 columns and 33960 nonzeros
Model fingerprint: 0x5019c038
Model 

   480   497 10174.5290  103  374 24289.5020 9403.19411  61.3%   121  264s
   626   667 11338.2020  134  389 26258.0985 10395.0858  60.4%   103  267s
   496   541 10326.0633  109  372 24289.5020 9403.19411  61.3%   133  269s
H  502   541                    24282.499937 9403.19411  61.3%   133  269s
H  509   541                    24239.649269 9403.19411  61.2%   132  269s
   540   585 10357.9483  116  377 24239.6493 9403.19411  61.2%   129  273s
   666   709 11362.9487  143  391 26258.0985 10395.0858  60.4%   101  275s
H  680   709                    26173.778163 10395.0858  60.3%   100  275s
H  584   635                    24211.788550 9403.19411  61.2%   125  279s
   634   682 10447.5873  135  377 24211.7886 9403.19411  61.2%   122  283s
   681   743 10490.2232  147  374 24211.7886 9403.19411  61.2%   118  288s
   742   772 10551.3725  156  378 24211.7886 9403.19411  61.2%   114  292s
   708   765 11442.9882  157  398 26173.7782 10395.0858  60.3%  99.0  297s
H  737   765             

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]